# Set up installs and imports







In [ ]:
#@title Install Libraries
# !pip install einops
# !pip install fitlog
# !pip install tsfresh
!pip install umap--learn
!pip install ipydatetime
!pip install --upgrade dash
!pip install jupyter_dash
!git clone https://github.com/imics-lab/time-series-label-assist.git

# ** TRISTAN MERGE **
# for dash video player component 
!pip install dash-player
# for uploading video to google drive
!pip install -U kora
!pip install dash-bootstrap-components
!pip install dash-bootstrap-templates
# **

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap--learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82830 sha256=eca534a492c00d3dfee6c792e3c7bdd1f849c80a28ad9ac05c46515d34ad5301
  Stored in directory: /root/.cache/pip/wheels/f4/3e/1c/596d0a463d17475af648688443fa4846fef624d1390339e7e9
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3-none-any.whl size=55509 sha256=06981f2dce008a22900fa23164773ee5e2691cf971a938e986cf2a5cf4b09b6a
  Stored in directory: /root/.cache/pip/wheels/b9/89/cc/59ab91ef5b21dc2ab3635528d7d227f49dfc9169905dcb959d
Successfully built umap--learn pynndescent
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/

In [ ]:
#@title Imports
import pandas as pd

import os
import shutil
from tabulate import tabulate # for verbose tables, showing data
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report, precision_recall_fscore_support
import urllib.request # to get files from web w/o !wget

import io
import sys
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
import ipydatetime
import IPython
import math
import csv
from keras.models import load_model
import numpy as np
from IPython.display import display
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from ipywidgets import FileUpload
import datetime
import time
from datetime import datetime
import random
import umap.umap_ as umap
import ipywidgets as widgets
import json
import matplotlib.pyplot as plt
import seaborn as sns
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash import ctx
from sklearn.preprocessing import LabelEncoder
from google.colab import files

# ** TRISTAN CHANGES ** 

# videoplayer
import dash_player
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
# GRAPH LAYER
from plotly.subplots import make_subplots
from google.colab import output
output.enable_custom_widget_manager()

import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template

# imports necessary for google drive upload
from kora.drive import upload_public
from google.colab import files

from ipywidgets import GridBox, Checkbox, HBox, VBox
from IPython.display import Markdown
from IPython.display import display
import re
# **

from dash.dependencies import Input, Output
import plotly.graph_objects as go

my_dir = os.getcwd()

#need to adjust paths for storing the data of the np arrays
os.chdir(os.path.join(my_dir, 'time-series-label-assist'))

working_dir = os.getcwd()

sys.path.append(os.path.join(my_dir, 'time-series-label-assist','data', 'arrays'))
sys.path.append(os.path.join(my_dir, 'time-series-label-assist', 'preprocessing'))
from preprocessing import split
from data_loaders import empatica
from prediction import model

## Import TWristAR & Gesture Phase Segmentation


In [ ]:
#@title Import TWristAR & Gesture Phase Segmentation
def get_py_file(fname, url):
    """checks for local file, if none downloads from URL.    
    :return: nothing"""
    #fname = 'load_data_utils.py'
    #ffname = os.path.join(my_dir,fname)
    if (os.path.exists(fname)):
        print ("Local",fname, "found, skipping download")
    else:
        print("Downloading",fname, "from IMICS git repo")
        urllib.request.urlretrieve(url, filename=fname)

get_py_file(fname = 'load_data_utils.py', url = 'https://raw.githubusercontent.com/imics-lab/load_data_time_series/main/load_data_utils.py')
get_py_file(fname = 'load_data_transforms.py', url = 'https://raw.githubusercontent.com/imics-lab/load_data_time_series/main/load_data_transforms.py')
get_py_file(fname = 'twristar_load_dataset.py', url = 'https://raw.githubusercontent.com/imics-lab/load_data_time_series/main/HAR/TWristAR/twristar_load_dataset.py')
get_py_file(fname = 'gesture_phase_segmentation_load_dataset.py', url = 'https://raw.githubusercontent.com/imics-lab/load_data_time_series/main/Gesturing_Signing/gesture_phase_segmentation_load_dataset.py')

In [ ]:
import load_data_transforms as xform
import load_data_utils as utils
import twristar_load_dataset as twristar
import gesture_phase_segmentation_load_dataset as gps

Local load_data_utils.py found, skipping download
Local load_data_transforms.py found, skipping download
Local load_data_utils.py found, skipping download
Local load_data_transforms.py found, skipping download


# Uploads

In [ ]:
#@title Select Dataset **PERSONAL ONLY WORKING RIGHT NOW**
dataset = "Personal" #@param ["TWristAR","TWristAR Free-Form","Gesture Phase Segmentation", "Personal"]

In [ ]:
#@title Upload CSV File
os.chdir(my_dir)
dataList_i = os.listdir(os.path.join(working_dir, 'data'))
labelingList_i = os.listdir(os.path.join(working_dir, 'labeling'))

if dataset == "TWristAR":
    ir1_dict = twristar.get_twristar_ir1_dict()
    label_dict = twristar.label_map_twristar['label']
    key_list = list(ir1_dict) # ir1_dict.keys() returns a dict_keys type
    my_key = 0 # sort of a global
if dataset == "TWristAR Free-Form":
    ir1_dict = twristar.get_twristar_ir1_dict(scripted=False)
    label_dict = twristar.label_map_twristar['label']
    key_list = list(ir1_dict) # ir1_dict.keys() returns a dict_keys type
    my_key = 0 # sort of a global
if dataset == "Gesture Phase Segmentation":
    ir1_dict = gps.get_gps_ir1_dict()
    label_dict = gps.label_map_gps['label']
    key_list = list(ir1_dict) # ir1_dict.keys() returns a dict_keys type
    my_key = 0 # sort of a global

if dataset == "Personal":
  print('Please input the path for the data file')
  os.chdir(os.path.join(working_dir,'data'))
  file_upload_d = files.upload()


  print('\nPlease input the path for the label file')
  os.chdir(os.path.join(working_dir, 'labeling'))
  file_upload_l = files.upload()


  #Find the symmetric difference between the original list of data items and the
  #new list including the uploaded items
  dataList_n = os.listdir(os.path.join(working_dir, 'data'))
  newItem_d = set(dataList_n).symmetric_difference(dataList_i)

  labelingList_n = os.listdir(os.path.join(working_dir, 'labeling'))
  newItem_l = set(labelingList_n).symmetric_difference(labelingList_i)


  if newItem_d:
      file_c = list(file_upload_d.keys())
      df = pd.read_csv(io.StringIO(file_upload_d[file_c[len(file_c) - 1]].decode('utf-8')), sep = ',')
  else:
      os.chdir(os.path.join(working_dir, 'data', 'testdata'))
      df = pd.read_csv('exampleFile.csv')

  if newItem_l:
      file_c = list(file_upload_l.keys())
      labelListDF = pd.read_csv(io.StringIO(file_upload_l[file_c[len(file_c) - 1]].decode('utf-8')), sep = ',')
  else:
      os.chdir(os.path.join(working_dir, 'data', 'testdata'))
      labelListDF = pd.read_csv('testLabelList.csv')

os.chdir(working_dir)

Please input the path for the data file



Please input the path for the label file


Saving TWristAR_label_list.csv to TWristAR_label_list.csv


In [ ]:
#@title Temp For Uploading Video & Save to Google Drive

# ** TRISTAN MERGE
# save copy of original dataframe
original_df = df.copy()

# ** IF YOU NEED TO UPLOAD FILE TO GOOGLE DRIVE FROM COLAB **
# uploads the file at that path to google drive
# upload_public returns the link of video.

# Run this line with path to video if you want to upload to google drive.
# url = upload_public('/content/selfsense/data/sub1_sit_stand.mp4')

# Hard coded for now so I don't flood my google drive with new downloads of ^
drive_link = 'https://drive.google.com/uc?id=1LmJ3fRtpX-vmrzEaXEUzlPpErXBhr6tB'


# ** IF YOU HAVE UPLOADED TO GOOGLE DRIVE, TAKE ID AND PUT AFTER 'https://drive.google.com/uc?id='
# E.g. https://drive.google.com/file/d/1NFdMUGmMxV_is9r17xzwPWodq_0cmJjW/view
# - This is link to view the file in Google Drive
# 1) ID is 1NFdMUGmMxV_is9r17xzwPWodq_0cmJjW
# 2) So video is at, https://drive.google.com/uc?id=1NFdMUGmMxV_is9r17xzwPWodq_0cmJjW

In [ ]:
#@title Upload Video & Save to Google Drive NEW UI
markdown_text1 = "There are two options for uploading a video: you can upload the video file directly to this Google Colab notebook using an upload widget, or provide a shared Google Drive link to the video."
display(Markdown(f"<h4>{markdown_text1}</h4>"))
markdown_text2 = "Would you like to upload the video file through an upload widget?"
display(Markdown(f"<h4>{markdown_text2}</h4>"))

def handle_yes(b):
  print('Please select your video file')
  os.chdir(os.path.join(working_dir,'data'))
  uploaded = files.upload()

  for file_name, content in uploaded.items():
      with open(file_name, 'wb') as f:
          f.write(content)
      file_path = os.path.abspath(file_name)
      print(f'\nFile {file_name} has been saved to {file_path}')
      
      markdown_text3 = "Would you like to upload that video to your Google Drive?"
      display(Markdown(f"<h4>{markdown_text3}</h4>"))

      def handle_yes2(b):
        global drive_link
        drive_link = upload_public(file_path)
        print("This is your Google Drive link: ", drive_link)
        return drive_link

      def handle_no2(b):
        global drive_link
        print("This is just for testing.")
        drive_link = 'https://drive.google.com/uc?id=1LmJ3fRtpX-vmrzEaXEUzlPpErXBhr6tB'
        return drive_link

      yes_button2 = widgets.Button(description='Yes')
      yes_button2.on_click(handle_yes2)

      no_button2 = widgets.Button(description='No')
      no_button2.on_click(handle_no2)

      buttons_container2 = widgets.HBox([yes_button2, no_button2])
      display(buttons_container2)

def handle_no(b):
    print("You must provide a Google Drive Link")
    print("An example of a shared Google Drive Link is: https://drive.google.com/file/d/1eMpeaMfS5jOkqLQNEfNALSxkJ59kRN8s/view?usp=sharing")
    text = widgets.Text()
    display(text)
    def on_submit(b):
        global drive_link
        drive_link = text.value
        # search for /d/ in link
        match = re.search("/d/([a-zA-Z0-9-_]+)", drive_link)

        if match:
            file_id = match.group(1)
            # provide new link
            drive_link = f'https://drive.google.com/uc?id={file_id}'
            print("This is your updated drive link: ", drive_link)
        else:
            print("Not a valid Google Drive link")
        return drive_link
    submit_button = widgets.Button(description='Submit')
    submit_button.on_click(on_submit)
    display(submit_button)

yes_button = widgets.Button(description='Yes')
yes_button.on_click(handle_yes)

no_button = widgets.Button(description='No')
no_button.on_click(handle_no)

buttons_container = widgets.HBox([yes_button, no_button])
display(buttons_container)

In [ ]:
#@title Creating Column List (Sensors we want to plot)
# make original copy of df
original_df = df

# make list of column headers, for plotting y values of Raw Time Series
cols = list(df)
# print(cols)

# if df has a confidence column, remove it from column list
if "confidence" in cols:
    pass
    # cols.remove('confidence')
# else, make a confidence column assigned an undefined
else:
    df['confidence'] = "Undefined"

# if df has a label column, remove it from column list
if "label" in cols:
    pass
    # cols.remove('label')
# else, make a label column assigned an undefined
else:
    df['label'] = "Undefined"

# info for UI
print("Only select the sensors you want plotted on the graph\n")

checkboxes = [Checkbox(value=False, description=label) for label in cols]
grid = GridBox(children=checkboxes, layout=widgets.Layout(grid_template_columns="repeat(4, 1fr)"))
display(grid)

# create submit button
submit_button = widgets.Button(description="Submit")
display(submit_button)

# Define a function to handle the button click
def handle_submit(sender):
  selected_data = []
  for i in range(0, len(checkboxes)):
      if checkboxes[i].value == True:
          selected_data = selected_data + [checkboxes[i].description]
  print("New cols list: ", selected_data)
  
  return selected_data

# Attach the function to the button
submit_button.on_click(handle_submit)

Only select the sensors you want plotted on the graph



GridBox(children=(Checkbox(value=False, description='datetime'), Checkbox(value=False, description='accel_x'),…

Button(description='Submit', style=ButtonStyle())

New cols list:  ['accel_x', 'accel_y', 'accel_z', 'accel_ttl']


In [ ]:
#@title Color List & Dictionary
# ** TRISTAN MERGE
# https://sashamaps.net/docs/resources/20-colors/

# Label color list & dictionary
colorList = ('#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe',
             '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', 
             '#000000')
# This assigns a specific color from colorList to each label in provided label list.
colorDict = dict(zip(labelListDF, colorList))

# Confidence list & dictionary
confidenceValues = ("High", "Medium", "Low", "Undefined")
confidenceColors = ('#3cb44b', '#ffe119', '#FF0000', '#000000')
confDict = dict(zip(confidenceValues, confidenceColors))

# # Cols color list & dictionary
# # This assigns a specific color from colorList to each label in provided label list.
# colsColorList = list(colorList)
# colsColorList.remove("#000000")
# random.shuffle(colsColorList)
# colsColorDict = dict(zip(cols, colsColorList))
# ** 


3. **Visualize raw time-series** 


---


Here we want an interactive plot. If the data are loaded as a continuous time series of dimensions (timesteps, channels), then the plot should visualize the entire time series.

In this interactive cell the user should have the functionality to be able to change the label an entire segment by providing the starting and ending timestamps and the new label.

# Manual Labeling

In [ ]:
#@title Manual Labeling Interface
variable_name = ""
labelList = list(labelListDF)
size = len(df)

# get user input data from cols interface
cols = handle_submit(submit_button)

# set layout
fig_layout = make_subplots(rows=3, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.1,
                    row_heights=[5,5,50],
                    subplot_titles=("Label Line", "Confidence Line", "Raw Time-Series Data"),
                    )
fig_layout.update_layout(height=10, width=1000)
# link x-axis for timeseries slider
fig_layout.update_xaxes(matches='x')
# theme stuff
load_figure_template("bootstrap")
fig_layout.update_layout(template="bootstrap")

# create UI fields for manually adding label & confidence level
manual_label_UI_fields = dbc.Card(
    [
      html.H4(children='''User Input Fields for Manually Adding a Label:'''),
      html.Div(children='''
        Format as YYYY-MM-DD HH:MM:SS E.g (2021-10-08 16:50:21)
      '''),
      html.Br(),
      
      html.Div([
          "Start Time: ",
          # Need to change ids, etc
          dcc.Input(id='start-input', 
                    type='text', 
                    placeholder = "YYYY-MM-DD HH:MM:SS"),
          dcc.Checklist(id='start-checkbox',
                    options=[{'label': 'Fill Start UI with Click', 'value': 'Checked'}],
                    value=['Unchecked']),
          html.Div(id='start-output'), 
          html.Div(id='start-cb-output'),
      ]),
      html.Br(),
      
      html.Div([
          "End   Time: ",
          # Need to change ids, etc
          dcc.Input(id='end-input',
                    type='text',
                    placeholder = "YYYY-MM-DD HH:MM:SS"),
          dcc.Checklist(id='end-checkbox',
                    options=[{'label': 'Fill End UI with Click', 'value': 'Checked'}],
                    value=['Unchecked']),
          html.Div(id='end-output'), 
          html.Div(id='end-cb-output'),
      ]),
      html.Br(),
      
      html.Div([
          "Labels: ",
          # Dummy values, need to get values from labelList csv
          dcc.Dropdown(labelList, 
                      placeholder = 'Select a Label',
                      id='label-selection'),
          # https://dash.plotly.com/dash-core-components/dropdown
          # this was for an output call back that prints curr value
          html.Div(id='label-output')
      ]),
      html.Br(),
      
      html.Div([
          "Degree of Confidence: ",
          dcc.Dropdown(["High", "Medium", "Low", "Undefined"], 
                      placeholder = 'Select a Confidence Level',
                      id='confidence-selection'),
          html.Div(id='confidence-output')
      ]),
      html.Br(),
      
      html.Button('Update Graph', id='btn-manual-label', n_clicks=0),
      html.Br(),
    ],
    body=True,
)

# create UI fields for video operations
video_UI_fields = dbc.Card(
    [
        # input field for offset
        html.H5(children='''Set Data/Video Offset:'''),
        html.Div(children='''Zero Offset: Both video and data start at zero.'''),
        html.Br(),
        html.Div(children='''Positive Offset: Data starts at zero, and video starts at offset."'''),
        html.Br(),
        html.Div(children='''Negative Offset: Data starts at offset, and video starts at zero."'''),
        html.Br(),
        dcc.Input(id='video-offset-input', type='text', placeholder = "Input in Seconds", value="0"),
        html.Div(id='vid-offset-output'),
        html.Br(),

        # sync video -> data
        html.H5(children='''Sync Video to Data:'''),
        html.Div(children='''Plot line on data graph at current time in video.'''),
        html.Button("Sync", id="button-sync-vid", n_clicks=0),
        html.Br(), html.Br(),


        # seek time input field
        html.H5(children='''Seek to Time in Video:'''),
        html.Div(children='''E.g. If you want to go to 2 minute time stamp in video, 
          input in seconds, or "120"'''),
        dcc.Input(id='seek-input', type='text', placeholder = "Input in Seconds"),
        html.Button("Seek", id="button-seek-to"),
        html.Div(id="div-current-time"),
        html.Div(html.Button('Reset Inputs', id='reset-button-2')),
        ],
    body=True,
)

# Makes list and assigns highest value to each index
labelLine = []
for i in range(size):
    labelLine.append(1)

def plotGraph():
    # Make figure for raw time series
    i = 0
    for element in cols:
      fig_layout.add_trace(go.Scatter(x=df['datetime'], y=df[cols[i]],
                      mode='lines', # 'lines' or 'markers'
                      name=cols[i]), row=3, col=1)
      i+=1
    fig_layout.update_layout(
        xaxis=dict(),#end xaxis  definition
        xaxis1_rangeslider_visible=False,
        xaxis2_rangeslider_visible=False,
        xaxis3_rangeslider_visible=True
        )
    # Count number of labels
    tempDf = df.ne(df.shift())
    labelCount = tempDf.loc[tempDf.label == True, 'label'].count()
    # pd.set_option('display.max_rows', None)  # or 1000
    # print(tempDf['label'])
    # print(labelCount)

    # Make two lists to store indices
    labelsStartIndex = []
    labelsEndIndex = []
    # Start index at -1 to match indices
    index = -1
    # Take size to put into endIndex array to show last index
    size = len(tempDf['label'])

    # Loop through column to find Start Indices
    for element in tempDf['label']:
        if element == True:
            index+=1
            labelsStartIndex.append(index)
        else:
            index+=1
    # print("Start Indices", labelsStartIndex)

    # Loop through Start Indices list and get the index before next change
    for element in labelsStartIndex:
        labelsEndIndex.append(element - 1)
    # Remove first so we dont get the garbage value
    labelsEndIndex.pop(0)
    # Append size of column because thats last known label index
    labelsEndIndex.append(size)
    # print("End Indices", labelsEndIndex)

    # line with labels
    i = 0
    size = len(labelsStartIndex)
    size = int(size)
    currLabel = df['label']
    for x in range(size):
        fig_layout.add_trace(go.Scatter(
            x = df.loc[labelsStartIndex[i]:labelsEndIndex[i],'datetime'],
            y = labelLine,
            mode="lines",
            name=currLabel.at[labelsStartIndex[i]],
            text=df.loc[labelsStartIndex[i]:labelsEndIndex[i],'label'],
            line_color=colorDict[currLabel.at[labelsStartIndex[i]]],
            textposition="top center",
            line_width=5,  # set the line width to 2
            showlegend=False
            ), row=1, col=1)
        i+=1

    # line with degree of confidence
    i = 0
    currConfidence = df['confidence']
    for x in range(size):
        fig_layout.add_trace(go.Scatter(
            x = df.loc[labelsStartIndex[i]:labelsEndIndex[i],'datetime'],
            y = labelLine,
            mode="lines",
            name=currConfidence.at[labelsStartIndex[i]],
            text=df.loc[labelsStartIndex[i]:labelsEndIndex[i],'confidence'],
            line_color=confDict[currConfidence.at[labelsStartIndex[i]]],
            textposition="top center",
            line_width=5,  # set the line width to 2
            showlegend=False
            ), row=2, col=1)
        i+=1

    fig_layout.update_layout(height=500, #width=1000,
                  legend_title_text="Sensors")
    fl_reset = go.Figure(fig_layout)
    # Reference website https://dash.plotly.com/layout
    # Build App
    external_stylesheets = [dbc.themes.BOOTSTRAP]
    load_figure_template("bootstrap")

    app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
    app.title = 'Manually Label Time-Series Data'
    
    app.layout = dbc.Container(
      [
          html.H2("Manually Label Raw Time-Series Data"),
          html.Hr(),
          # html.Div(children='''
          #   Dash: A web application framework for your data.
          # '''),
          html.Br(),
          
          # data
          dbc.Row(
              [
                  dbc.Col(manual_label_UI_fields, md=4),
                  # dbc.Col(dcc.Graph(id='graph-output',figure={}), md=8),
                  dbc.Col(dbc.Card([
                      dcc.Graph(id='graph-output',figure={})
                  ]), 
                  md=8),
              ],
              align="center", style={"height": "rem"}
            ),
       
          # video
          dbc.Row(
              [
                  dbc.Col(video_UI_fields, md=4),
                  dbc.Col(dbc.Card([
                      dash_player.DashPlayer(id = "video-player", url=drive_link, controls=True)
                  ])
                  , md=4),
                  # dbc.Col(dash_player.DashPlayer(id = "video-player", url=url, controls=True), md=8),
                  html.Br(),
                  # current vid vals
                  html.Hr(),
                  html.Div(id='vid-sync-plot-dt-output'),
                  html.Div(id='vid-sync-plot-offset-output'),
                  html.Div(id='vid-sync-plot-timestamp-output'),               
              ],
              align="center",
          ),
      ],
      fluid=True,
    )
    
    #####################################################
    # callback for user input start time
    @app.callback(
        Output("start-output", "children"),
        Input('start-input', "value")
    )
    def update_start_time(start_value):
        update_start_time.data = start_value
        return "Start Value: {}".format(start_value)
    
    # callback for user input end time
    @app.callback(
        Output("end-output", "children"),
        Input('end-input', "value")
    )
    def update_end_time(end_value):
        update_end_time.data = end_value
        return "End Value: {}".format(end_value)
    
    # callback for user input label selection
    @app.callback(
        Output("label-output", "children"),
        Input('label-selection', "value")
    )
    def update_label(label_value):
        update_label.data = label_value
        return "Label Selection: {}".format(label_value)
    
    # callback for user input confidence selection
    @app.callback(
        Output("confidence-output", "children"),
        Input('confidence-selection', "value")
    )
    def update_confidence_degree(confidence_value):
        update_confidence_degree.data = confidence_value
        return "Degree of Confidence: {}".format(confidence_value)
    
    # callback for user input video offset
    @app.callback(
        Output("vid-offset-output", "children"),
        Input('video-offset-input', "value")
    )
    def update_video_offset(videoOffset):
        update_video_offset.data = videoOffset
        return "Offset Value: {}".format(videoOffset)

    # callback for printing current time under video
    @app.callback(
        Output("div-current-time", "children"),
        Input("video-player", "currentTime")
    )
    def update_time(currentTime):
        update_time.data = currentTime
        return "Current Timestamp of Video: {}".format(currentTime)

    @app.callback(
        Output('graph-output', 'figure'),
        Output('vid-sync-plot-dt-output', 'children'),
        Output('vid-sync-plot-offset-output', 'children'),
        Output('vid-sync-plot-timestamp-output', 'children'),
        [Input('btn-manual-label', 'n_clicks'),
        Input('button-sync-vid', 'n_clicks'),]
    )
    def updateGraph(btn1, btn2):
        # Initial render of graph
        # What we update when adding new labels
        updatedFig = fl_reset
        vid_to_data_sync = 0
        offset = 0
        timestamp = 0
        # If btn is clicked
        if "btn-manual-label" == ctx.triggered_id:
          start = update_start_time.data
          end = update_end_time.data
          label = update_label.data
          confidence = update_confidence_degree.data

          # Initialize a copy of df to manipulate
          label_df = df.copy()

          # Removes any decimals in datetime column, can assume label last at least a second
          # 2021-10-08 16:50:21.000000000 -> 16:50:21
          label_df['datetime'] = pd.to_datetime(label_df['datetime']).dt.strftime('%Y-%m-%d %H:%M:%S')

          # using strf converts datetime to string
          # using strp converts string to datetime
          """
          start = datetime.strftime(start, '%Y-%m-%d %H:%M:%S')
          end = datetime.strftime(end, '%Y-%m-%d %H:%M:%S')
          print(type(start), type(end))
          """
          
          # find start & end within 'datetime' column
          # Find start and end within 'datetime' column
          # Get Start indices and shove into list, First indice is where first label needs to go
          startIndices = label_df.index[label_df['datetime'] == start].tolist()
          startIndex = startIndices[0]
          # print("Starting Index", startIndex)
          # Get End indices and shove into list, Last indice is where labels need to end
          endIndices = label_df.index[label_df['datetime'] == end].tolist()
          endIndex = endIndices[-1]
          # print("Ending Index", endIndex)
          
          # Within df, look at 'label' column & assign label to range of startIndex to endIndex
          label_df.loc[startIndex:endIndex, 'label'] = label

          # label_df.label has our updated label column
          # Replaces original label column with the newly updated label column
          df.loc[:, 'label'] = label_df.label
          
          # replace confidence value inbetween start and end index
          label_df.loc[startIndex:endIndex, 'confidence'] = confidence
          df.loc[:, 'confidence'] = label_df.confidence
          # Check updated original dataframe's label column to see if updated
          # print(df['label'])
          
          # **** LABEL LINE **** #

          # Count number of labels
          tempDf = df.ne(df.shift())
          labelCount = tempDf.loc[tempDf.label == True, 'label'].count()
          # pd.set_option('display.max_rows', None)  # or 1000
          # print(tempDf['label'])
          # print(labelCount)

          # Make two lists to store indices
          labelsStartIndex = []
          labelsEndIndex = []
          # Start index at -1 to match indices
          index = -1
          # Take size to put into endIndex array to show last index
          size = len(tempDf['label'])

          # Loop through column to find Start Indices
          for element in tempDf['label']:
              if element == True:
                  index+=1
                  labelsStartIndex.append(index)
              else:
                  index+=1
          # print("Start Indices", labelsStartIndex)

          # Loop through Start Indices list and get the index before next change
          for element in labelsStartIndex:
              labelsEndIndex.append(element - 1)
          # Remove first so we dont get the garbage value
          labelsEndIndex.pop(0)
          # Append size of column because thats last known label index
          labelsEndIndex.append(size)
          # print("End Indices", labelsEndIndex)

          # Pandas iloc or whatever to plot between datetime ranges [0,31] on first iteration of loop
          # line with labels
          i = 0
          size = len(labelsStartIndex)
          size = int(size)
          currLabel = df['label']
          
          # Update the graph
          updatedFig = go.Figure(fl_reset)
          
          # plot label line
          for x in range(size):
              updatedFig.add_trace(go.Scatter(
                  x = df.loc[labelsStartIndex[i]:labelsEndIndex[i],'datetime'],
                  y = labelLine,
                  mode="lines",
                  name=currLabel.at[labelsStartIndex[i]],
                  text=df.loc[labelsStartIndex[i]:labelsEndIndex[i],'label'],
                  line_color=colorDict[currLabel.at[labelsStartIndex[i]]],
                  textposition="top center",
                  line_width=5,  # set the line width to 2
                  showlegend=False
                  ), row=1, col=1)
              i+=1

          # line with degree of confidence
          i = 0
          currConfidence = df['confidence']
          for x in range(size):
              updatedFig.add_trace(go.Scatter(
                  x = df.loc[labelsStartIndex[i]:labelsEndIndex[i],'datetime'],
                  y = labelLine,
                  mode="lines",
                  name=currConfidence.at[labelsStartIndex[i]],
                  text=df.loc[labelsStartIndex[i]:labelsEndIndex[i],'confidence'],
                  line_color=confDict[currConfidence.at[labelsStartIndex[i]]],
                  textposition="top center",
                  line_width=5,  # set the line width to 2
                  showlegend=False
                  ), row=2, col=1)
              i+=1

        if "button-sync-vid" == ctx.triggered_id:
            # updatedFig = fig

            # # create copy df to modify
            temp_df = df.copy()

            # Assume all datetime are in column named 'datetime'
            # Take first & last datetime from column
            firstDateTime = temp_df['datetime'][0]
            lastDateTime = temp_df['datetime'][len(temp_df['datetime']) - 1]

            firstDateTime = pd.to_datetime(firstDateTime)
            lastDateTime = pd.to_datetime(lastDateTime)

            # Convert first & last values to datetime, w/ unix specifications
            # cut down decimal
            firstDateTime = firstDateTime.strftime('%Y-%m-%d %H:%M:%S')
            # have to convert back to datetime
            firstDateTime = pd.to_datetime(firstDateTime)
            # unix specifications
            firstDateTime = pd.to_datetime(firstDateTime, unit='s', origin='unix')
            # cut down decimal
            lastDateTime = lastDateTime.strftime('%Y-%m-%d %H:%M:%S')
             # have to convert back to datetime
            lastDateTime = pd.to_datetime(lastDateTime)
            # unix specifications
            lastDateTime = pd.to_datetime(lastDateTime, unit='s', origin='unix')

            # Get unix time for first and last (datetime -> unix)
            firstUnixTime = (time.mktime(firstDateTime.timetuple()))
            lastUnixTime = (time.mktime(lastDateTime.timetuple()))

            # offset & timestamp retrieval 
            # get from update_video_offset.data, strp by casting as int
            offset = int(update_video_offset.data)
            # get from update_time.data, , strp by casting as int
            timestamp = int(update_time.data)

            # if our offset is negative
            # be able to start data before its recorded
            if offset < 0:
              vid_to_data_sync = firstUnixTime + timestamp + abs(offset)

            else:
              # # get actual time after math in unix
              vid_to_data_sync = firstUnixTime + timestamp - offset
            # # # unix -> datetime, and strf down to whole second
            vid_to_data_sync = pd.to_datetime(vid_to_data_sync, unit='s', origin='unix')
            vid_to_data_sync = vid_to_data_sync.strftime('%Y-%m-%d %H:%M:%S')

            # reset graph
            updatedFig = go.Figure(fl_reset)

            # plot at that x a red line
            updatedFig.add_vline(
              # hard coded range to highlight, change these x's to video time
              x=vid_to_data_sync, line_width=2, line_dash="dash", line_color="red"
            )
            # label added at same x value
            updatedFig.add_annotation(x=vid_to_data_sync, text="Video Sync", font=dict(size=15, color="white"),align="center")
              
        return updatedFig, "Sync Datetime: {}".format(vid_to_data_sync), "Sync Offset: {}".format(offset), "Sync Timestamp: {}".format(timestamp)
    
    # callback for seeking to timestamp in video from user inputted value
    @app.callback(
        Output("video-player", "seekTo"), 
        [Input("button-seek-to", "n_clicks"),
        Input('seek-input', 'value')]
    )
    def set_seekTo(n_clicks, seek_value):
        if 'button-seek-to' == ctx.triggered_id:
          return seek_value

    # callback for resetting video UI
    @app.callback(
        [Output('video-offset-input', 'value'),Output('seek-input', 'value')],
        [Input('reset-button-2', 'n_clicks')])
    def reset_input(n_clicks):
        if n_clicks:
            return '0','0'
        else:
            return '0','0'
    
    # checkbox
    @app.callback(Output('start-cb-output', 'children'), [Input('start-checkbox', 'value')])
    def start_ui_fill(value):
        if len(value) == 2:
            start_ui_fill.data = value
            return "Start time fill toggled on"
        else:
            start_ui_fill.data = value
            return "Start time fill toggled off"

    # fill start UI with clicked data
    @app.callback(
        Output('start-input', 'value'),
        Input('graph-output', 'clickData'))
    def st_x_value(clickData):
        try:
          if start_ui_fill.data.count('Checked') == 1:
            xval1 = clickData['points'][0]['x']
            st_x_value.data = xval1
            dt_val1 = xval1.split(".")[0]
            return str(dt_val1)
          else:
            return update_start_time.data
        except Exception as e:
          pass

    # checkbox
    @app.callback(Output('end-cb-output', 'children'), [Input('end-checkbox', 'value')])
    def end_ui_fill(value):
        if len(value) == 2:
            end_ui_fill.data = value
            return "End time fill toggled on"
        else:
            end_ui_fill.data = value
            return "End time fill toggled off"

    # fill start UI with clicked data
    @app.callback(
        Output('end-input', 'value'),
        Input('graph-output', 'clickData'))
    def ed_x_value(clickData):
        try:
          if end_ui_fill.data.count('Checked') == 1:
            xval2 = clickData['points'][0]['x']
            ed_x_value.data = xval2
            dt_val2 = xval2.split(".")[0]
            return str(dt_val2)
          else:
            return update_end_time.data
        except Exception as e:
          pass

    app.run_server(mode='external')
plotGraph()

New cols list:  ['accel_x', 'accel_y', 'accel_z', 'accel_ttl']
Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
#@title check label/confidence changes

# displays whole df
# pd.set_option('display.max_columns', None)  # or 1000
# pd.set_option('display.max_rows', None)  # or 1000
# pd.set_option('display.max_colwidth', None)  # or 199

# displays section of df
pd.set_option('display.max_columns', 1000)  # or 1000
pd.set_option('display.max_rows', 1000)  # or 1000
pd.set_option('display.max_colwidth', 199)  # or 199
df

# Auto Labeling

In [ ]:
#@title Specification for Length of Timesteps
info = widgets.Label('Specifications for length of timesteps: i.e. (highest_frequency * seconds)')
display(info)
window_size= widgets.IntText(
    value=96,
    description='Window Size:',
    disabled=False
)
freq = widgets.Output()
display(window_size, freq)


steps = widgets.IntText(
    value= 96,
    description='Steps:',
    disabled=False
)
step = widgets.Output()
display(steps, step)

Label(value='Specifications for length of timesteps: i.e. (highest_frequency * seconds)')

IntText(value=96, description='Window Size:')

Output()

IntText(value=96, description='Steps:')

Output()

In [ ]:
#@title Build Model
cnn = model.CNN()
newDf = df.copy()
newDf = newDf.drop('confidence',axis = 1)
# gesture data set
# Drop the 'story' column if it exists
if 'story' in newDf.columns:
    newDf = newDf.drop('story', axis=1)

newDf = newDf.set_index('datetime')

print("Available Labels: ", list(labelListDF))
print("Encoding labels present in the data")
new_np = split.TimeSeriesNP(window_size.value, steps.value)
new_np.setArrays(newDf, one_hot_encode=False ,labels =labelList)

print("x shape is: "+ str(new_np.x.shape))
print("y shape is: " + str(new_np.y.shape))

x_train, x_validation, y_train, y_validation = train_test_split(new_np.x, new_np.y.ravel(), test_size = 0.25)
cnn.only_train_data(x_train, x_validation, y_train, y_validation )

info = widgets.Label('Name your model if you want to save it')
display(info)
modelName = widgets.Text(
    value = "model",
    placeholder='1D_CNN',
    description='Model Name:',
    disabled=False
)
q = widgets.Output()
display(modelName,q)


#credit: https://stackoverflow.com/questions/58716248/how-to-create-a-simple-button-with-output-in-a-python-jupyter-notebook
info = widgets.Label('This button will construct and train the model')
display(info)

def clicked(arg):
    print("Please wait while model is built.")
    cnn.build()
    cnn.train()
    cnn.model.summary()

button_download = widgets.Button(description = 'Build')   
button_download.on_click(clicked)
display(button_download)

Available Labels:  ['Downstairs', 'Jogging', 'Sitting', 'Standing', 'Upstairs', 'Walking', 'Undefined']
Encoding labels present in the data
x shape is: (118, 96, 4)
y shape is: (118,)


/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



Label(value='Name your model if you want to save it')

Text(value='model', description='Model Name:', placeholder='1D_CNN')

Output()

Label(value='This button will construct and train the model')

Button(description='Build', style=ButtonStyle())

In [ ]:
#@title Save Model
from typing_extensions import ClassVar
import csv
def clicked(arg):
    os.chdir(os.path.join(working_dir, 'prediction', 'models'))
    cnn.model.save( modelName.value + ".h5")

    os.chdir(os.path.join(working_dir, 'prediction', 'dimensions'))
    f = open(modelName.value + ".csv", 'w')
    print("Please wait while the model is being saved.")
    writer = csv.writer(f)
    writer.writerow([window_size.value, steps.value])
    f.close()
    print("Model Saved.")


info = widgets.Label('This button will save the model')
display(info)
button_download = widgets.Button(description = 'Save Model')   
button_download.on_click(clicked)
display(button_download)
os.chdir(working_dir)

Label(value='This button will save the model')

Button(description='Save Model', style=ButtonStyle())

In [ ]:
#@title Load Model + Upload Data Set for Auto Labeling
radiobut = True
model_select = widgets.RadioButtons(
        options=[''],
        value= '',
        description='Models:',
        disabled=False
    )

if os.path.isdir(os.path.join(working_dir,"prediction","models")):
    dir_list = os.listdir(os.path.join(working_dir, "prediction", "models"))
    model_select.options = dir_list

else:
    radiobut = False
    model_select = widgets.Text(
    value='',
    placeholder='Type something',
    description='File Path:',
    disabled=False
)
info = widgets.Label('Select or Upload Model')
display(info)
freq = widgets.Output()
display(model_select,freq)




os.chdir(os.path.join(working_dir, 'data'))
print('Upload dataset you wish to have labeled\n')
uploaded = files.upload()
file_c = list(uploaded.keys())
to_label_df = pd.read_csv(io.StringIO(uploaded[file_c[len(file_c) - 1]].decode('utf-8')), sep = ',')
os.chdir(working_dir)

Label(value='Select or Upload Model')

RadioButtons(description='Models:', options=('original_GPS_model.h5', 'TWristARmodel.h5', 'model.h5', 'GPS_mod…

Output()

Upload dataset you wish to have labeled



Saving sub1_free_form_walk.csv to sub1_free_form_walk.csv


In [ ]:
#@title UMAP Dash App
if radiobut:
    modelPath = os.path.join(working_dir,"prediction","models", model_select.value)
    new_model = load_model(modelPath)
    os.chdir(working_dir)

elif(os.path.isfile(model_select.value)):
    new_model = load_model(model_select.value)

testModel = model.CNN()
testModel.setModel(new_model)

#new dictionary for altering labels
label_num_dict = {new_np.mapping[k] : k for k in new_np.mapping}
#use text labels to create hover text for each point on the scatter plot
#user needs to know what labels correspond to which points
text_labels = []
for i in range(len(new_np.y)):
    text_labels.append(new_np.mapping[new_np.y[i]])
def nearestNeighbor(embedding, pointIndex, y):
    min_o =  math.inf
    min_c = math.inf
    nearest_neighbor = None
    point_loc = embedding[pointIndex]
    n_color = pointIndex
    for i in range(len(embedding)):
        distance = pow(pow(point_loc[0] - embedding[i][0], 2) + pow(point_loc[1] - embedding[i][1] , 2), 0.5)
        if distance < min_c and distance != 0 and y[i] == y[pointIndex]:
            n_color = i
            min_c = distance
        if distance < min_o and distance != 0:
            n_overall = i
            min_o = distance
    return n_overall, n_color

load_figure_template("bootstrap")
npInput = new_np.y
testModel.only_test_data(new_np.x, new_np.y)
predictions = testModel.model.predict(new_np.x, verbose=0, batch_size = 32)


reducer = umap.UMAP(n_neighbors = 15, n_components =2)
embedding = reducer.fit_transform(predictions)
plotly_umap = px.scatter(embedding, x=embedding[:,0], y=embedding[:,1], color = npInput, 
                         range_color=[0, len(labelList)], hover_name=text_labels,color_continuous_scale=px.colors.sequential.Jet)
plotly_umap.update_layout(
    title = "2D UMAP", xaxis_title="X", yaxis_title="Y"
)

graph1 = px.scatter()
graph2 = px.scatter()
graph3 = px.scatter()
lineGraph = px.line(df, x = "datetime", y = cols)

def layout():
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
    umap_app = JupyterDash(__name__,external_stylesheets=external_stylesheets)
    umap_app.title = 'UMAP'
    
    umap_app.layout = html.Div([
        html.H1("UMAP"),
        dcc.Graph(
            id='umap-graph',
            figure=plotly_umap
            ),
        html.Br(),
        html.Div([
            dcc.Markdown("""
                **Click Data**

                Click on markets in the graph.
            """),
        ]),        
        html.Div([
        html.Div([
            html.Div([
                dcc.Dropdown(labelList, '', id='dropdown')
            ], className="three columns"),

            html.Div([
                html.Button('Add Label', id='button', n_clicks=0)
            ], className= "three columns"),
        ], className="row")
            
        ]),
        html.Br(),
        html.Div([
            dcc.Markdown("""
            **Time-Series Plot**

            """),
            dcc.Graph(
                id='plot-clicked',
                figure = lineGraph
            ),
        ], ),

        html.Br(),
        html.Div([
            html.Div([
                dcc.Graph(id='graph1', figure=px.line())
            ], className="four columns"),

            html.Div([
                dcc.Graph(id='graph2', figure=px.line())
            ], className= "four columns"),

            html.Div([
                dcc.Graph(id='graph3', figure=px.line())
            ], className="four columns"),
        ], className="row"),
        dcc.Store(id='store_data', data = None, storage_type='memory')
    ])
    @umap_app.callback(
        [Output('umap-graph', 'figure'),
         Output('plot-clicked', 'figure'),
         Output('graph1', 'figure'),
         Output('graph2', 'figure'),
         Output('graph3', 'figure'),
         Output('store_data', 'data')],
        Input('umap-graph', 'clickData'),
        Input('button', 'n_clicks'),
        [State("dropdown", "value"),
         State("store_data","data")]
        )
    def update_app(clickData, n_clicks, value, data):

        #need to have some blank copies of graphs to avoid errors upon initial loading when nothing has been clicked
        umap = px.scatter(embedding, x=embedding[:,0], y=embedding[:,1], color = npInput,  
                          range_color=[0, len(labelList)], hover_name=text_labels, color_continuous_scale=px.colors.sequential.Jet)
        plot = px.line(df, x = "datetime", y = cols)
        graph1 = px.line()
        graph2 = px.line()
        graph3 = px.line()
        id = None
        if "umap-graph" == ctx.triggered_id:
            data = clickData["points"][0]
            id = data['pointIndex']
            near_o, near_c = nearestNeighbor(embedding, id, npInput)
            umap = px.scatter(embedding, x=embedding[:,0], y=embedding[:,1], color = npInput, range_color=[0,len(labelList)], hover_name=text_labels, color_continuous_scale=px.colors.sequential.Jet)

            #add trace over selected point
            umap.add_trace(
                go.Scatter(
                    mode='lines+markers+text',
                    x=[embedding[id][0]],
                    y=[embedding[id][1]],
                    marker=dict(
                        color='grey',
                        size=10,
                        symbol = "circle",
                    ),
                    name = "Selected Point",
                    showlegend=True
                )
            )

            #add point on UMAP for nearest overall neighbor
            umap.add_trace(
                go.Scatter(
                    mode='lines+markers+text',
                    # text=["Nearest Neighbor Overall"],
                    # textposition = 'top center',
                    x=[embedding[near_o][0]],
                    y=[embedding[near_o][1]],
                    marker=dict(
                        color='red',
                        size=10,
                        symbol = "diamond",
                        line=dict(
                            color='black',
                            width=2
                        )
                    ),
                    name = "Nearest Neighbor: Overall",
                    showlegend=True
                )
            )



            #highlight segment of time-series for selected point
            plot = px.line(df, x = "datetime", y = cols)
            plot.add_vrect(
              x0= new_np.time[id][0], x1= new_np.time[id][1],
              fillcolor="grey", opacity=0.5,
              layer="below", line_width=0,
            )

            plot.add_vrect(
              x0= new_np.time[near_o][0], x1= new_np.time[near_o][1],
              fillcolor="red", opacity=0.5,
              layer="below", line_width=0,
            )

            #modify subgraph
            g_1 = df.loc[df["datetime"].between(new_np.time[id][0], new_np.time[id][1])]
            graph1 = px.line(g_1, x = "datetime", y = cols)
            graph1.update_layout(title = "Selected Point", showlegend=False,xaxis_title="")

            g_2 = df.loc[df["datetime"].between(new_np.time[near_o][0], new_np.time[near_o][1])]
            graph2 = px.line(g_2, x = "datetime", y = cols)
            graph2.update_layout(title = "Nearest Neighbor: Overall",showlegend=False,xaxis_title="")

            graph3 = px.line()
            if near_o != near_c:
                umap.add_trace(
                    go.Scatter(
                        mode='lines+markers+text',
                        # text=["Nearest Neighbor of Same Type"],
                        # textposition = 'top center',
                        x=[embedding[near_c][0]],
                        y=[embedding[near_c][1]],
                        marker=dict(
                            color='purple',
                            size=10,
                            symbol = "diamond",
                            line=dict(
                                color='black',
                                width=2
                            )
                        ),
                    name = "Nearest Neighbor: Same Type",
                    showlegend=True
                    )
                )
                
                plot.add_vrect(
                    x0= new_np.time[near_c][0], x1= new_np.time[near_c][1],
                    fillcolor="purple", opacity=0.5,
                    layer="below", line_width=0,
                )

                g_3 = df.loc[df["datetime"].between(new_np.time[near_c][0], new_np.time[near_c][1])]
                graph3 = px.line(g_3, x = "datetime", y = cols)
                graph3.update_layout(title = "Nearest Neighbor: Same Type", showlegend=False,xaxis_title="")
            else:
                graph3.update_yaxes(visible=False)
                graph3.update_xaxes(visible=False)

        if "button" == ctx.triggered_id:
            # and data != None and value != ""
            npInput[data] =  label_num_dict[value]
            text_labels[data] = value
            umap = px.scatter(embedding, x=embedding[:,0], y=embedding[:,1], color = npInput, range_color=[0,len(labelList)], hover_name=text_labels, color_continuous_scale=px.colors.sequential.Jet)

        return umap,plot,graph1,graph2,graph3,id
    umap_app.run_server(mode='external')
layout()


Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
#@title Rewrite numpy class to fix (TEMP)
#all modules that Lee Hinkle used in his code. Included to cover all bases 
import time
from time import gmtime, strftime, localtime #for displaying Linux UTC timestamps in hh:mm:ss
from datetime import datetime
from datetime import timedelta
import numpy as np
#imports for computing and displaying output metrics
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


class TimeSeriesNP():
    def __init__(self, time_steps, step):
        self.step = step
        self.time_steps = time_steps

    def timeSlice(self, df):

        #I am doing it this way because in the future the user should be able to 
        #enter a data set that does not contain subject and label columns without
        #breaking the functionality
        N_FEATURES = len(df.columns)
        not_features = 0
        if 'label' in df.columns:
            not_features+=1
        if 'sub' in df.columns:
            not_features+=1
        N_FEATURES-= not_features  

        #subtract 2 for labels and subject columns
        #use seperate arrays for each set of values
        segments = []
        labels = []
        subject = []
        times = []
        steps = 0
        relevantColumns = df.columns[:-not_features]

        #step through the data set and only take 
        for i in range(0, len(df) - self.time_steps, self.step):
            df_segX = df[relevantColumns].iloc[i: i + self.time_steps]
            df_lbl = df['label'].iloc[i: i + self.time_steps]
            df_sub = df['sub'].iloc[i: i + self.time_steps]
            
            # Save only if labels are the same for the entire segment and valid
            # if (df_lbl.value_counts().iloc[0] != self.time_steps):
            #     labels[i] = df_lbl.iloc[i]["label"]

            subject.append(df['sub'].iloc[i])
            labels.append(df['label'].iloc[i]) 
            segments.append(df_segX.to_numpy())   
            times.append([df.index[i], df.index[i + self.time_steps]])
                

        # Bring the segments into a better shape, convert to nparrays
        reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, self.time_steps, N_FEATURES)
        labels = np.asarray(labels)
        subject = np.asarray(subject)
        times = np.asarray(times)
        # both labels and sub are row arrays, change to single column arrays
        labels = labels[np.newaxis].T
        subject = subject[np.newaxis].T
        # check for nan - issue with resampled data
        bad_data_locations = np.argwhere(np.isnan(reshaped_segments))
        np.unique(bad_data_locations[:,0]) #[:,0] accesses just 1st column
        if (bad_data_locations.size!=0):
            print("Warning: Output arrays contain NaN entries")
        return reshaped_segments, labels, subject, times
    

    def setArrays(self, df, labels, encode = True, one_hot_encode= True):
        self.x, self.y, self.sub, self.time = self.timeSlice(df)

        #encode using integer or one-hot endcoding
        if encode:
            y_vector = np.ravel(self.y) #encoder won't take column vector
            le = LabelEncoder()
            integer_encoded = le.fit_transform(labels)
            self.mapping = dict(zip(range(len(le.classes_)), le.classes_))
            
            if (one_hot_encode):
                onehot_encoder = OneHotEncoder(sparse=False)
                integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
                onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
                self.y=onehot_encoded
            else:
                self.y = le.transform(self.y)
    

    def updateArrays(self, df):

        #take existing arrays and simply stack the newer ones on the bottom
        temp_x, temp_y, temp_sub, temp_time = timeslice(df)
        self.x = np.vstack([self.x, temp_x])
        self.y = np.vstack([self.y, temp_y])
        self.sub = np.vstack([self.sub, temp_sub])
        self.time = np.vstack([self.time, temp_time])
    
    def finalize(self):

        #final step used in Lee Hinkle's implementation
        #not entirely sure if it necessary
        self.x = np.delete(self.x, (0), axis=0) 
        self.y = np.delete(self.y, (0), axis=0) 
        self.sub = np.delete(self.sub, (0), axis=0)
        self.sub = sub.astype(int) # convert from float to int
    

    """
    use a dictionary to pass in the splitting of the subjects.
    should be useful for experiments but requires more testing
    to ensure proper functionality.
    
    """
    def trainTestSplit_subj(self,
        verbose = True,
        incl_xyz_accel = False, # include component accel_x/y/z in ____X data
        incl_rms_accel = True, # add rms value (total accel) of accel_x/y/z in ____X data
        incl_val_group = False, # split train into train and validate
        split_subj = dict
                    (train_subj = [1,2],
                    validation_subj = [11,21], 
                    test_subj = [3]) 
        ):

        sub_num = np.ravel(self.sub[ : , 0] ) # convert shape to (1047,)
        print(sub_num)
        if (not incl_val_group):
            train_index = np.nonzero(np.isin(sub_num, split_subj['train_subj'] + 
                                            split_subj['validation_subj']))
            x_train = self.x[train_index]
            y_train = self.y[train_index]
        else:
            train_index = np.nonzero(np.isin(sub_num, split_subj['train_subj']))           
            x_train = self.x[train_index]
            y_train = self.y[train_index]

            validation_index = np.nonzero(np.isin(sub_num, split_subj['validation_subj']))
            x_validation = self.x[validation_index]
            y_validation = self.y[validation_index]

        test_index = np.nonzero(np.isin(sub_num, split_subj['test_subj']))
        x_test = self.x[test_index]
        y_test = self.y[test_index]
        if (incl_val_group):
            return x_train, y_train, x_validation, y_validation, x_test, y_test
        else:
            return x_train, y_train, x_test, y_test


            if(verbose):
                headers = ("Reshaped data","shape", "object type", "data type")
                mydata = [("x_train:", x_train.shape, type(x_train), x_train.dtype),
                        ("y_train:", y_train.shape ,type(y_train), y_train.dtype),
                        ("x_test:", x_test.shape, type(x_test), x_test.dtype),
                        ("y_test:", y_test.shape ,type(y_test), y_test.dtype)]
                print(tabulate(mydata, headers=headers))

            return x_train, y_train, x_test, y_test

    def trainTestSplit(self, validation, testSize):
        x_train, x_test, y_train, y_test= sklearn.train_test_split(self.x, self.y, test_size = testSize, random_state = 1)

        if validation:
            x_train, x_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.25, random_state=1)
            return x_train, y_train, x_validation, y_validation, x_test, y_test
        else:
            return x_test, x_train, y_test, y_train

In [ ]:
#@title Predict Labels
"""
citations:
getting name of a file without the extension:       https://theprogrammingexpert.com/python-remove-extension/
interesting techniques for labeling df:             https://www.geeksforgeeks.org/label-based-indexing-to-the-pandas-dataframe/
fill in values of df according to dictionary:       https://stackoverflow.com/questions/64481108/assign-a-value-between-index-values-ranges-in-pandas

"""

#this one works
def getNPdimensions(modelname, fileloc):
    filename = modelname[:len(modelname) - 3]
    filename = filename + ".csv"
    print(filename)
    path = os.path.join(fileloc, filename)

    np_dimensions = []
    if os.path.isfile(path):
        fileList = pd.read_csv(path)
        np_dimensions = list(fileList)
    np_dimensions[0] = int(math.floor(float(np_dimensions[0])))
    np_dimensions[1] = int(math.floor(float(np_dimensions[1])))

    return np_dimensions
###################################################################



if radiobut:
    previousPath = os.getcwd()
    modelPath = os.path.join(working_dir,"prediction","models", model_select.value)
    new_model = load_model(modelPath)
    os.chdir(previousPath)

elif(os.path.isfile(model_select.value)):
    new_model = load_model(model_select.value)

dfA = to_label_df
# SPECIFIC TO DR. NGU'S DATASET
dfA = dfA.rename(columns={'p_accelerometer_time' : 'datetime'})
dfA['sub'] = "Undefined"
dfA = dfA.set_index('datetime')
###################################################################

np_dimensions = getNPdimensions(model_select.value , os.path.join(working_dir,"prediction","dimensions"))
np_labeling = TimeSeriesNP(96,32)
np_labeling.setArrays(dfA, labelList)

cnn = model.CNN()
testModel = model.CNN()
testModel.setModel(new_model)
testModel.only_test_data(np_labeling.x, np_labeling.y)


####################################################################
def label_confidence(l):
    return max(l)/sum(l)

def pred_to_labels(predictions):
    y_pred = np.argmax(predictions, axis=-1)
    y_pred_labels = np.vectorize(labels_dict.get)(y_pred)
    return y_pred_labels

def flag_sus_labels(npObject, labels_dict, labels, model, CON_THRESHOLD):
    predictions = model.predict(npObject.x, verbose = 0, batch_size = 32)
    y_pred = np.argmax(predictions, axis=-1)
    y_pred_labels = np.vectorize(labels_dict.get)(y_pred)
    unique, counts = np.unique(y_pred_labels, return_counts=True)
    
    fig, ax = plt.subplots(1,1,figsize=(6,3))
    ax.plot(y_pred,linestyle="",marker=".")
    plt.title('Predicted Activity Labels')
    ax.set_xlabel('Elapsed Time(seconds)')

    ax.set_yticks(np.arange(len(labels)))
    ax.set_yticklabels(labels)
    ax.grid()
    for i in range(0, len(predictions), 20):
        if sum([label_confidence(j)<CON_THRESHOLD for j in predictions[i:i+20]]) >0:
            plt.axvspan(i, i+20, color='gold', alpha=0.2)

    plt.tight_layout() # keeps labels from being cutoff when saving as pdf
    plt.show()
    return predictions

def labelDf(df, labels_dict, model, npObject):
    print(df)
    print(labels_dict)
    print("Model input shape")
    print(model.input_shape)
    print("numpy x shape")
    print(npObject.x.shape)
    #copy the dataframe and initialize all predicted labels to Undefined
    labeledDf = df
    labeledDf['pred_labels'] = "Undefined"
    
    #reformat y_pred to hold the string values for labels
    predictions = model.predict(npObject.x, verbose = 0, batch_size = 32)
    y_pred = np.argmax(predictions, axis=-1)
    y_pred_labels = np.vectorize(labels_dict.get)(y_pred)
    unique, counts = np.unique(y_pred_labels, return_counts=True)

    #print out relevant information about labels and dictionary
    print("Final Predicted Label Counts")
    print (np.asarray((unique, counts)).T)
    #label the new dataframe
    
    for z in range(y_pred_labels.size):
        start = npObject.time[z][0]
        end = npObject.time[z][1]
        labeledDf.loc[start : end, ['pred_labels']] = y_pred_labels[z]
    
    return predictions

def confusion_mat(npObject, model, labelLoc):
    labels_dict = npObject.mapping

    predictions = model.model.predict(npObject.x, verbose = 0, batch_size = 32)
    y_pred = np.argmax(predictions, axis=-1)
    y_pred_labels = np.vectorize(labels_dict.get)(y_pred)
    unique, counts = np.unique(y_pred_labels, return_counts=True)

    test_acc , y_pred = model.run()
    y_test_act = np.argmax(npObject.y, axis=-1)
    class_rpt_for_log = classification_report(y_test_act, y_pred, target_names = unique)
    print(class_rpt_for_log)

    cm = confusion_matrix(y_test_act, y_pred)
    cm_df = pd.DataFrame(cm, index = unique, columns = unique)
    fig = plt.figure(figsize=(9,8))
    sns.heatmap(cm_df, annot=True, fmt='d', cmap= 'cubehelix_r')
    plt.title('1D CNN Accuracy:{0:.3f}'.format(accuracy_score(y_test_act, y_pred)))
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()

info = widgets.Label('This button will add predicted labels to the dataset using the model you selected')
display(info)
def clicked(arg):
    predictions = labelDf(dfA, np_labeling.mapping, testModel.model, np_labeling)
    #flag_sus_labels(new_np, labels_to_dict(os.path.join(os.getcwd(), labelList)),os.path.join(os.getcwd(), 'testLabelList.csv'), testModel.model, 0.99)
    #confusion_mat(new_np ,testModel, os.path.join(os.getcwd(), labelList))
button_download = widgets.Button(description = 'Add Labels')   
button_download.on_click(clicked)
display(button_download)

TWristARmodel.csv


/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning:

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.



Label(value='This button will add predicted labels to the dataset using the model you selected')

Button(description='Add Labels', style=ButtonStyle())

                                accel_x   accel_y   accel_z  accel_ttl  \
datetime                                                                 
2019-11-24 19:59:00.000000000  0.046875 -0.015625  0.984375  -0.014386   
2019-11-24 19:59:00.031249920  0.046875 -0.031250  1.000000   0.001586   
2019-11-24 19:59:00.062500096  0.046875 -0.031250  1.000000   0.001586   
2019-11-24 19:59:00.093750016  0.046875 -0.031250  1.000000   0.001586   
2019-11-24 19:59:00.124999936  0.046875 -0.031250  0.984375  -0.014014   
...                                 ...       ...       ...        ...   
2019-11-24 20:10:59.500000000  0.281250  0.125000  0.953125   0.001586   
2019-11-24 20:10:59.531249920  0.265625  0.109375  0.953125  -0.004527   
2019-11-24 20:10:59.562500096  0.265625  0.109375  0.953125  -0.004527   
2019-11-24 20:10:59.593750016  0.296875  0.109375  0.953125   0.004263   
2019-11-24 20:10:59.750000128  0.296875  0.109375  0.953125   0.004263   

                                   la

# Review Auto Labels


In [ ]:
#@title Specification for Confidence Threshold
info1 = widgets.Label('If the predicted probability for a label is above this threshold, it will be considered a valid label. If it falls below this threshold, it will be flagged as suspect.')
display(info1)

info2 = widgets.Label('Enter value in range of 0-1.')
display(info2)

def on_button_clicked(b):
    conf_thresh = float(text_ui.value)
    print("Confidence Threshold is ", conf_thresh)
    return conf_thresh

description = widgets.Label("Enter Confidence Threshold: ")
text_ui = widgets.FloatText(value = 0.9)
button = widgets.Button(description="Submit")
button.on_click(on_button_clicked)

# Use HBox layout to place the text field and button side by side
hbox = widgets.HBox([description,text_ui])
display(hbox)
display(button)

Label(value='If the predicted probability for a label is above this threshold, it will be considered a valid l…

Label(value='Enter value in range of 0-1.')

Button(description='Submit', style=ButtonStyle())

Confidence Threshold is  0.9


In [ ]:
#@title Functions for plotting label line and confidence line
# for dfA
# currently only have label line
# fig_ll_rawTS = make_subplots(rows=2, cols=1,
#                     shared_xaxes=True,
#                     vertical_spacing=0.1,
#                     row_heights=[10, 200],
#                     subplot_titles=("Label Line", "Raw Time Series"),
#                     )
# #fig_ll_rawTS.update_layout(height=200)
# load_figure_template("bootstrap")

# fix dfA to work in app
if dfA.columns[-1] ==  "pred_labels":
    dfA['label'] = dfA['pred_labels']
    dfA = dfA.drop('pred_labels', axis = 1)

if dfA.index.name != "datetime":
    dfA = dfA.set_index('datetime')

#print(dfA)
np_pred = split.TimeSeriesNP(*np_dimensions)
np_pred.setArrays(dfA, one_hot_encode=False ,labels = labelList)
npInput = np_pred.y

#new dictionary for altering labels
label_num_dict = {np_pred.mapping[k] : k for k in np_pred.mapping}
#use text labels to create hover text for each point on the scatter plot
#user needs to know what labels correspond to which points
text_labels = []
for i in range(len(np_pred.y)):
    text_labels.append(np_pred.mapping[np_pred.y[i]])
clear_output()
dfA = dfA.reset_index()

def plotSubplotLabelLineRawTS():
    fig_ll_rawTS = make_subplots(rows=2, cols=1,
                        shared_xaxes=True,
                        vertical_spacing=0.1,
                        row_heights=[20, 200],
                        subplot_titles=("Label Line", "Raw Time Series"),
                        )

    # Count number of labels
    tempDf = dfA.ne(dfA.shift())
    labelCount = tempDf.loc[tempDf.label == True, 'label'].count()
    # pd.set_option('display.max_rows', None)  # or 1000
    # print(tempDf['label'])
    # print(labelCount)

    # Make two lists to store indices
    labelsStartIndex = []
    labelsEndIndex = []
    # Start index at -1 to match indices
    index = -1
    # Take size to put into endIndex array to show last index
    size = len(tempDf['label'])

    # Loop through column to find Start Indices
    for element in tempDf['label']:
        if element == True:
            index+=1
            labelsStartIndex.append(index)
        else:
            index+=1
    # print("Start Indices", labelsStartIndex)

    # Loop through Start Indices list and get the index before next change
    for element in labelsStartIndex:
        labelsEndIndex.append(element - 1)
    # Remove first so we dont get the garbage value
    labelsEndIndex.pop(0)
    # Append size of column because thats last known label index
    labelsEndIndex.append(size)
    # print("End Indices", labelsEndIndex)

    # line with labels
    i = 0
    size = len(labelsStartIndex)
    size = int(size)
    currLabel = dfA['label']
    for x in range(size):
        fig_ll_rawTS.add_trace(go.Scatter(
            x = dfA.loc[labelsStartIndex[i]:labelsEndIndex[i],'datetime'],
            y = labelLine,
            mode="lines",
            name=currLabel.at[labelsStartIndex[i]],
            text=dfA.loc[labelsStartIndex[i]:labelsEndIndex[i],'label'],
            line_color=colorDict[currLabel.at[labelsStartIndex[i]]],
            textposition="top center",
            line_width=5,
            showlegend=False
            ), row=1, col=1)
        i+=1

    # Make figure for raw time series
    i = 0
    for element in cols:
      fig_ll_rawTS.add_trace(go.Scatter(x=dfA['datetime'], y=dfA[element],
                      mode='lines', # 'lines' or 'markers'
                      name=element), row=2, col=1)
      i+=1

    return fig_ll_rawTS

In [ ]:
#@title Flag on raw time series after callback
conf_thresh = on_button_clicked(button)

# append confidence column to dfA **NEW**
dfA_temp = dfA.copy()
dfA_temp["confidence"] = False
figt = px.scatter(x=dfA_temp["datetime"], y=dfA_temp["confidence"])

for i in range(len(predictions)):
    if max(predictions[i]) < conf_thresh:
        #print(max(predictions[i]))
        start = np_pred.time[i][0]
        end = np_pred.time[i][1]
        dfA_temp.loc[(dfA["datetime"] >= start) & (dfA_temp["datetime"] <= end), "confidence"] = True 
        figt.add_vrect(
            x0 = start,
            x1 = end,
            fillcolor="yellow",
            opacity = 0.5,
            layer = "below",
        )
figt.update_traces(y=dfA_temp["confidence"])
#figt.show()

testModel = model.CNN()
testModel.setModel(new_model)
testModel.only_test_data(np_pred.x, np_pred.y)

reducer = umap.UMAP(n_neighbors = 15, n_components =2)
embedding = reducer.fit_transform(predictions)

graph1 = px.scatter()
graph2 = px.scatter()
graph3 = px.scatter()
lineGraph = plotSubplotLabelLineRawTS()

for i in range(len(predictions)):
    if max(predictions[i]) < conf_thresh:
        #print(max(predictions[i]))
        start = np_pred.time[i][0]
        end = np_pred.time[i][1]
        lineGraph.add_vrect(
            x0 = start,
            x1 = end,
            fillcolor="yellow",
            opacity = 0.5,
            layer = "below",
            line_width = 0
        )

load_figure_template("bootstrap")
def layout():
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
    review_app = JupyterDash(__name__,external_stylesheets=external_stylesheets,)
    review_app.title = 'Review Flagged Labels'
    
    review_app.layout = html.Div([
        html.H3("Review Flagged Labels"),
        html.Br(),
        
        dcc.Markdown("**Video**"),
        # Video
        html.Div(className="row", children=[
            # video col 1
            html.Div(className="four columns", children=[
                dash_player.DashPlayer(id="video-player", url=drive_link, controls=True),
                html.Div(id='vid-sync-plot-dt-output'),
                html.Div(id='vid-sync-plot-offset-output'),
                html.Div(id='vid-sync-plot-timestamp-output'),
            ]),

            # UI column 2
            html.Div(className="four columns", style={"padding-left": "200px"}, children=[
                dcc.Markdown("""
                    **Set Data/Video Offset:**

                    Zero Offset: Both video and data start at zero.

                    Positive Offset: Data starts at zero, and video starts at offset.

                    Negative Offset: Data starts at offset, and video starts at zero.
                """),
                dcc.Input(id='video-offset-input', type='text', placeholder="Input in Seconds", value="0"),
                html.Div(id='vid-offset-output'),
                html.Br(),

                dcc.Markdown("""
                    **Sync Video to Data:**

                    Plot line on data graph at current time in video.
                """),
                html.Button("Sync", id="button-sync-vid", n_clicks=0),
                html.Br(), html.Br(),
            ]),

            # UI column 2
            html.Div(className="four columns", children=[
                dcc.Markdown("""
                    **Seek to Time in Video:**

                    E.g. If you want to go to 2 minute time stamp in video, 
                      input in seconds, or "120
                """),
                dcc.Input(id='seek-input', type='text', placeholder="Input in Seconds"),
                html.Button("Seek", id="button-seek-to"),
                html.Div(id="div-current-time"),
                html.Br(),
                html.Div(html.Button('Reset Inputs', id='reset-button-2')),
                html.Br(),
            ])
        ]),
        html.Br(),

        # RAW TIME SERIES plot and 3 umap point plots
        html.Div([
            dcc.Markdown("""
            **Time-Series Plot**
            """),
            dcc.Graph(
                id='plot-clicked',
                figure = lineGraph
            ),
        ], ),

        html.Br(),
        html.Div([
            html.Div([
                dcc.Graph(id='graph1', figure=px.line())
            ], className="four columns"),

            html.Div([
                dcc.Graph(id='graph2', figure=px.line())
            ], className= "four columns"),

            html.Div([
                dcc.Graph(id='graph3', figure=px.line())
            ], className="four columns"),
        ], className="row"),
        dcc.Store(id='store_data', data = None, storage_type='memory'),

      # UI for manual label of RAW TS
      html.Div([
        dcc.Markdown("""
          **User Input Fields for Manually Adding a Label:**

          Format as YYYY-MM-DD HH:MM:SS E.g (2021-10-08 16:50:21)
        """),
      ]),     
      html.Br(),
      html.Div(className="row", children=[
          # ui col 1
          html.Div(className="four columns", children=[
              html.Div([
                  "Start Time: ",
                  # Need to change ids, etc
                  dcc.Input(id='start-input', 
                            type='text', 
                            placeholder = "YYYY-MM-DD HH:MM:SS"),
                  dcc.Checklist(id='start-checkbox',
                                options=[{'label': 'Fill Start UI with Click', 'value': 'Checked'}],
                                value=['Unchecked']),
                  html.Div(id='start-output'), 
                  html.Div(id='start-cb-output'),
              ]),
              html.Br(),
              html.Div([
                  "End   Time: ",
                  # Need to change ids, etc
                  dcc.Input(id='end-input',
                            type='text',
                            placeholder = "YYYY-MM-DD HH:MM:SS"),
                  dcc.Checklist(id='end-checkbox',
                                options=[{'label': 'Fill End UI with Click', 'value': 'Checked'}],
                                value=['Unchecked']),
                  html.Div(id='end-output'), 
                  html.Div(id='end-cb-output'),
              ]),
              html.Br(),
          ]),
          
          # ui col 2
          html.Div(className="four columns", children=[
              html.Div([
                  "Labels: ",
                  # Dummy values, need to get values from labelList csv
                  dcc.Dropdown(labelList, 
                              placeholder = 'Select a Label',
                              id='label-selection'),
                  # https://dash.plotly.com/dash-core-components/dropdown
                  # this was for an output call back that prints curr value
                  html.Div(id='label-output')
              ]),
              html.Br(),
              html.Div([
                  "Flag: ",
                  dcc.Dropdown(["True", "False"], 
                              placeholder = 'Select T (flag) or F (unflag)',
                              id='flag-selection'),
                  html.Div(id='flag-output')
              ]),
          ])
      ]),
      html.Button('Update Graph', id='btn-manual-label', n_clicks=0),
      html.Br(), html.Br(),

      # UMAP
      html.Div(className="row", children=[
          # UMAP col1,
          html.Div(className="eight columns", children=[
              dcc.Markdown("**UMAP**"),
              dcc.Graph(
                  id='umap-graph',
                  figure=plotly_umap
              )
          ]),

          # UMAP UI, col 2
          html.Div(className="four columns", children=[
              html.Div([
                  dcc.Markdown("""
                      **Click Data**

                      Click on markets in the graph.
                  """),
              ]),
              html.Br(),
              html.Div([
                  html.Div([
                      dcc.Dropdown(labelList, '', id='dropdown')
                  ], className="three columns"),

                  html.Div([
                      html.Button('Add Label', id='button', n_clicks=0)
                  ], className= "three columns"),
              ], className="row")
          ])
      ]),
    ])

    ############################################################################
    # callback for user input start time
    @review_app.callback(
        Output("start-output", "children"),
        Input('start-input', "value")
    )
    def update_start_time(start_value):
        update_start_time.data = start_value
        return "Start Value: {}".format(start_value)
    
    # callback for user input end time
    @review_app.callback(
        Output("end-output", "children"),
        Input('end-input', "value")
    )
    def update_end_time(end_value):
        update_end_time.data = end_value
        return "End Value: {}".format(end_value)
    
    # callback for user input label selection
    @review_app.callback(
        Output("label-output", "children"),
        Input('label-selection', "value")
    )
    def update_label(label_value):
        update_label.data = label_value
        return "Label Selection: {}".format(label_value)
    
    # callback for user input flag selection
    @review_app.callback(
        Output("flag-output", "children"),
        Input('flag-selection', "value")
    )
    def update_flag(flag_value):
        update_flag.data = flag_value
        return "Flag: {}".format(flag_value)

    # umap & raw ts plot callbacks
    @review_app.callback(
        [Output('umap-graph', 'figure'),
         Output('plot-clicked', 'figure'),
         Output('graph1', 'figure'),
         Output('graph2', 'figure'),
         Output('graph3', 'figure'),
         Output('store_data', 'data')],
        Input('umap-graph', 'clickData'),
        Input('button', 'n_clicks'),
        # added
        Input('btn-manual-label', 'n_clicks'),

        [State("dropdown", "value"),
         State("store_data","data")]
        )
    def update_app(clickData, n_clicks, clicks, value, data):

        #need to have some blank copies of graphs to avoid errors upon initial loading when nothing has been clicked
        umap = px.scatter(embedding, x=embedding[:,0], y=embedding[:,1], color = npInput,  
                          range_color=[0, len(labelList)], hover_name=text_labels, color_continuous_scale=px.colors.sequential.Jet)
        plot = plotSubplotLabelLineRawTS()
        for i in range(len(predictions)):
            if max(predictions[i]) < conf_thresh:
                #print(max(predictions[i]))
                start = np_pred.time[i][0]
                end = np_pred.time[i][1]
                plot.add_vrect(
                    x0 = start,
                    x1 = end,
                    fillcolor="yellow",
                    opacity = 0.5,
                    layer = "below",
                    line_width = 0
                )
        graph1 = px.line()
        graph2 = px.line()
        graph3 = px.line()
        id = None
        if "umap-graph" == ctx.triggered_id:
            data = clickData["points"][0]
            id = data['pointIndex']
            near_o, near_c = nearestNeighbor(embedding, id, npInput)
            umap = px.scatter(embedding, x=embedding[:,0], y=embedding[:,1], color = npInput, range_color=[0,len(labelList)], hover_name=text_labels, color_continuous_scale=px.colors.sequential.Jet)

            #add trace over selected point
            umap.add_trace(
                go.Scatter(
                    mode='lines+markers+text',
                    x=[embedding[id][0]],
                    y=[embedding[id][1]],
                    marker=dict(
                        color='grey',
                        size=10,
                        symbol = "circle",
                    ),
                    name = "Selected Point",
                    showlegend=True
                )
            )

            #add point on UMAP for nearest overall neighbor
            umap.add_trace(
                go.Scatter(
                    mode='lines+markers+text',
                    # text=["Nearest Neighbor Overall"],
                    # textposition = 'top center',
                    x=[embedding[near_o][0]],
                    y=[embedding[near_o][1]],
                    marker=dict(
                        color='red',
                        size=10,
                        symbol = "diamond",
                        line=dict(
                            color='black',
                            width=2
                        )
                    ),
                    name = "Nearest Neighbor: Overall",
                    showlegend=True
                )
            )



            #highlight segment of time-series for selected point
            plot = plotSubplotLabelLineRawTS()
            for i in range(len(predictions)):
                if max(predictions[i]) < conf_thresh:
                    #print(max(predictions[i]))
                    start = np_pred.time[i][0]
                    end = np_pred.time[i][1]
                    plot.add_vrect(
                        x0 = start,
                        x1 = end,
                        fillcolor="yellow",
                        opacity = 0.5,
                        layer = "below",
                        line_width = 0,
                    )
            plot.add_vrect(
              x0= np_pred.time[id][0], x1= np_pred.time[id][1],
              fillcolor="grey", opacity=0.5,
              layer="below", line_width=0,
            )

            plot.add_vrect(
              x0= np_pred.time[near_o][0], x1= np_pred.time[near_o][1],
              fillcolor="red", opacity=0.5,
              layer="below", line_width=0,
            )

            #modify subgraph
            g_1 = dfA.loc[dfA["datetime"].between(np_pred.time[id][0], np_pred.time[id][1])]
            graph1 = px.line(g_1, x = "datetime", y = cols)
            graph1.update_layout(title = "Selected Point", showlegend=False,xaxis_title="")

            g_2 = dfA.loc[dfA["datetime"].between(np_pred.time[near_o][0], np_pred.time[near_o][1])]
            graph2 = px.line(g_2, x = "datetime", y = cols)
            graph2.update_layout(title = "Nearest Neighbor: Overall",showlegend=False,xaxis_title="")

            graph3 = px.line()
            if near_o != near_c:
                umap.add_trace(
                    go.Scatter(
                        mode='lines+markers+text',
                        # text=["Nearest Neighbor of Same Type"],
                        # textposition = 'top center',
                        x=[embedding[near_c][0]],
                        y=[embedding[near_c][1]],
                        marker=dict(
                            color='purple',
                            size=10,
                            symbol = "diamond",
                            line=dict(
                                color='black',
                                width=2
                            )
                        ),
                    name = "Nearest Neighbor: Same Type",
                    showlegend=True
                    )
                )
                
                plot.add_vrect(
                    x0= new_np.time[near_c][0], x1= new_np.time[near_c][1],
                    fillcolor="purple", opacity=0.5,
                    layer="below", line_width=0,
                )

                g_3 = dfA.loc[df["datetime"].between(new_np.time[near_c][0], new_np.time[near_c][1])]
                graph3 = px.line(g_3, x = "datetime", y = cols)
                graph3.update_layout(title = "Nearest Neighbor: Same Type", showlegend=False,xaxis_title="")
            else:
                graph3.update_yaxes(visible=False)
                graph3.update_xaxes(visible=False)

        if "button" == ctx.triggered_id:
            # and data != None and value != ""
            npInput[data] =  label_num_dict[value]
            text_labels[data] = value
            umap = px.scatter(embedding, x=embedding[:,0], y=embedding[:,1], color = npInput, range_color=[0,len(labelList)], hover_name=text_labels, color_continuous_scale=px.colors.sequential.Jet)
                # If update graph btn is clicked
        
        if "btn-manual-label" == ctx.triggered_id:
          start = update_start_time.data
          end = update_end_time.data
          label = update_label.data
          confidence = update_flag.data

          # Initialize a copy of dfA to manipulate
          label_df = dfA.copy()

          # Removes any decimals in datetime column, can assume label last at least a second
          # 2021-10-08 16:50:21.000000000 -> 16:50:21
          label_df['datetime'] = pd.to_datetime(label_df['datetime']).dt.strftime('%Y-%m-%d %H:%M:%S')

          # using strf converts datetime to string
          # using strp converts string to datetime
          """
          start = datetime.strftime(start, '%Y-%m-%d %H:%M:%S')
          end = datetime.strftime(end, '%Y-%m-%d %H:%M:%S')
          print(type(start), type(end))
          """
          
          # find start & end within 'datetime' column
          # Find start and end within 'datetime' column
          # Get Start indices and shove into list, First indice is where first label needs to go
          startIndices = label_df.index[label_df['datetime'] == start].tolist()
          startIndex = startIndices[0]
          # print("Starting Index", startIndex)
          # Get End indices and shove into list, Last indice is where labels need to end
          endIndices = label_df.index[label_df['datetime'] == end].tolist()
          endIndex = endIndices[-1]
          # print("Ending Index", endIndex)
          
          # Within df, look at 'label' column & assign label to range of startIndex to endIndex
          label_df.loc[startIndex:endIndex, 'label'] = label

          # label_df.label has our updated label column
          # Replaces original label column with the newly updated label column
          dfA.loc[:, 'label'] = label_df.label
          
          # replace confidence value inbetween start and end index
          label_df.loc[startIndex:endIndex, 'confidence'] = confidence
          dfA.loc[:, 'confidence'] = label_df.confidence
          # Check updated original dataframe's label column to see if updated
          # print(df['label'])
          
          # Update the graph
          plot = plotSubplotLabelLineRawTS()
          for i in range(len(predictions)):
              if max(predictions[i]) < conf_thresh:
                  #print(max(predictions[i]))
                  start = np_pred.time[i][0]
                  end = np_pred.time[i][1]
                  plot.add_vrect(
                      x0 = start,
                      x1 = end,
                      fillcolor="yellow",
                      opacity = 0.5,
                      layer = "below",
                      line_width = 0,
                  )

        return umap,plot,graph1,graph2,graph3,id

    review_app.run_server(mode='external')
layout()

Confidence Threshold is  0.9
Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



Dash app running on:


<IPython.core.display.Javascript object>